### Model Testing Figures for Paper

In [ ]:
%load_ext autoreload
%autoreload 2

import copy
import os

os.environ["PATH"] = "/usr/local/texlive/2025/bin/x86_64-linux:" + os.environ["PATH"]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from synference import SBI_Fitter, create_uncertainty_models_from_EPOCHS_cat

plt.style.use("paper.style")
model_name = "BPASS_DenseBasis_v4_final"
extra = "nsf_0"

bands = [
    "F435W",
    "F606W",
    "F775W",
    "F814W",
    "F850LP",
    "F090W",
    "F115W",
    "F150W",
    "F200W",
    "F277W",
    "F335M",
    "F356W",
    "F410M",
    "F444W",
]
hst_bands = ["F435W", "F606W", "F775W", "F814W", "F850LP"]
new_band_names = [
    (f"HST/ACS_WFC.{band.upper()}" if band in hst_bands else f"JWST/NIRCam.{band.upper()}")
    for band in bands
]
data_err_file = "/home/tharvey/work/JADES-DR3-GS_MASTER_Sel-F277W+F356W+F444W_v13.fits"


fitter = SBI_Fitter.load_saved_model(
    f"/home/tharvey/work/synference/models/{model_name}/{model_name}_{extra}_posterior.pkl",
    grid_path="/home/tharvey/work/synference/grids/grid_BPASS_Chab_DenseBasis_SFH_0.01_z_14_logN_6.0_Calzetti_v4_multinode.hdf5",
)


empirical_noise_models = create_uncertainty_models_from_EPOCHS_cat(
    data_err_file,
    bands,
    new_band_names,
    plot=False,
    hdu="OBJECTS",
    save=False,
    min_flux_error=0,
    model_class="asinh",
)

fitter.feature_array_flags["empirical_noise_models"] = empirical_noise_models

Figure 2: Calibration and recovery.

In [ ]:
fitter.plot_coverage()

## Calculate TARP

In [ ]:
import tarp

sample_path = (
    f"/home/tharvey/work/synference/models/{model_name}/plots/{extra}/posterior_samples.npy"
)
true = fitter._y_test

samples = np.load(sample_path)


out = tarp.get_tarp_coverage(
    samples,
    true,
    norm=True,
    bootstrap=True,
    num_bootstrap=100,
)


## Load metrics

In [ ]:
metrics = f"/home/tharvey/work/synference/models/{model_name}/{model_name}_{extra}_metrics.json"

import json

with open(metrics, "r") as f:
    met = json.load(f)

In [ ]:
fitter.fitted_parameter_names


param_labels = [
    r"$\log M_*$",
    r"$\log Z/Z_\odot$",
    r"$\rm \log_{10} \ A_V$",
    r"$\log$ SFR",
    "$t_{25}$",
    "$t_{50}$",
    "$t_{75}$",
    r"$\log$ Age$_M$",
    r"$\log$ SFR$_{10}$",
    r"$\log M_{*,surv}$",
    "$\\beta$",
]

In [ ]:
fitter.fitted_parameter_names

In [ ]:
fitter.plot_parameter_deviations(snr_bins=[5, 10, np.inf])

In [ ]:
snr_indices = fitter.bin_noisy_testing_data(
    X_test=fitter._X_test,
    y_test=fitter._y_test,
    snr_bins=[5, np.inf],
    snr_feature_names=None,
    return_indices=True,
)[0]

snr_indices = np.array(snr_indices).flatten()

# Make it a boolean mask
snr_indices = np.isin(np.arange(fitter._y_test.shape[0]), snr_indices)

In [ ]:
points = fitter._X_test[snr_indices]

In [ ]:
# import gridspec

fig = plt.figure(
    figsize=(9, 9),
    dpi=150,
)

gs = fig.add_gridspec(3, 6, height_ratios=[1, 1, 1.44])

ax_top_l = fig.add_subplot(gs[0, :2])
ax_top_r = fig.add_subplot(gs[0, 2:4])
ax_top_c = fig.add_subplot(gs[0, 4:6])
ax_middle_l = fig.add_subplot(gs[1, :2])
ax_middle_r = fig.add_subplot(gs[1, 2:4])
ax_middle_c = fig.add_subplot(gs[1, 4:6])
ax1 = fig.add_subplot(gs[2, :3])
ax2 = fig.add_subplot(gs[2, 3:])
axs = [ax_top_l, ax_top_r, ax_top_c]


# TARP and Coverage
ax1.plot([0, 1], [0, 1], "k--", label="Ideal", alpha=0.5)
ax1.set_xlabel("Predicted Percentile")
ax1.set_ylabel("Empirical Percentile")
ax1.text(0.05, 0.90, "TARP", transform=ax1.transAxes, fontsize=16)

ax2.plot([0, 1], [0, 1], "k--", label="Ideal", alpha=0.5)
ax2.set_xlabel("Credibility Level")
ax2.set_ylabel("Expected Coverage")
ax2.text(0.05, 0.90, "SBC", transform=ax2.transAxes, fontsize=16)

ax_params = [
    "log_mass",
    "log10_floor_sfr_10",
    "log10_Av",
    "log10_mass_weighted_age",
    "beta",
    "log_surviving_mass",
]
param_latex = [
    r"\log(\rm M_\star/M_\odot)",
    r"\log(\rm SFR_{10 \rm Myr}/ M_\odot yr^{-1})",
    r"\log(\rm A_V / \rm mag)",
    r"\log(\rm Age_M / \rm yr)",
    r"\beta",
    r"\rm \log M_{\star,surv}/M_\odot)",
]
indices = [list(fitter.fitted_parameter_names).index(p) for p in ax_params]

# ax_top_r.set_xscale('log')
# ax_top_r.set_yscale('log')
# ax_top_r.set_xlim(1e-4, 1e3)
# ax_top_r.set_ylim(1e-4, 1e3)
# TARP

ecp, alpha = out[0], out[1]
ecp_mean = np.mean(ecp, axis=0)
ecp_std = np.std(ecp, axis=0)
ax1.plot(alpha, ecp_mean, label="TARP", color="b")
ax1.fill_between(alpha, ecp_mean - ecp_std, ecp_mean + ecp_std, alpha=0.5, color="b")
ax1.fill_between(alpha, ecp_mean - 2 * ecp_std, ecp_mean + 2 * ecp_std, alpha=0.2, color="b")


# SBC
trues = fitter._y_test
import copy

ptrues = copy.deepcopy(trues)[snr_indices]
psamples = copy.deepcopy(samples)[:, snr_indices, :]

# lower SNR
ltrues = copy.deepcopy(trues)[~snr_indices]
lsamples = copy.deepcopy(samples)[:, ~snr_indices, :]

print(ptrues.shape, trues.shape, psamples.shape, samples.shape)


# Apply SNR mask to samples and trues

ndata, npars = trues.shape
ranks = (samples < trues[None, ...]).sum(axis=0)

unicov = [np.sort(np.random.uniform(0, 1, ndata)) for j in range(200)]
unip = np.percentile(unicov, [5, 16, 84, 95], axis=0)

cdf = np.linspace(0, 1, len(ranks))
cmap = plt.get_cmap("RdYlBu")
colors = [cmap(i / npars) for i in range(npars)]

for i in range(npars):
    xr = np.sort(ranks[:, i])
    xr = xr / xr[-1]
    ax2.plot(cdf, cdf, "k--")

    ax2.plot(xr, cdf, lw=2, label=param_labels[i], alpha=1, color=colors[i])
ax2.set(adjustable="box", aspect="equal")
ax1.set(adjustable="box", aspect="equal")

ax2.set_xlabel("Predicted Percentile")
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.legend(fontsize=8)

for i, ax in enumerate([ax_top_l, ax_top_r, ax_top_c, ax_middle_l, ax_middle_r, ax_middle_c]):
    param = ax_params[i]
    idx = indices[i]
    order_index = list(fitter.fitted_parameter_names).index(param)

    # r2 = met["R_squared"][order_index]
    # rmse = met["RMSE"][order_index]
    # Recalculate RMSE and R2 for just high SNR data
    from sklearn.metrics import mean_squared_error, r2_score

    y_true = ptrues[:, idx]
    y_pred = np.median(psamples[:, :, idx], axis=0)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    # Get 16/50/84 percentiles
    # p16 = np.percentile(samples[:, :, idx], 16, axis=0)
    # p50 = np.percentile(samples[:, :, idx], 50, axis=0)
    # p84 = np.percentile(samples[:, :, idx], 84, axis=0)
    # ax.errorbar(true[:, idx], p50, yerr=[p50 - p16, p84 - p50], fmt='o',
    #  alpha=0.1, color='C0', markersize=2)
    # Instead, plot as hexbin and count all draws
    # broadcast true to match samples shape
    # Do Hexbins of LOW SNR data in blues
    ltrue_full = np.repeat(ltrues[:, idx][np.newaxis, :], lsamples.shape[0], axis=0).flatten()
    lyy = lsamples[:, :, idx].flatten()

    true_full = np.repeat(ptrues[:, idx][np.newaxis, :], psamples.shape[0], axis=0).flatten()
    yy = psamples[:, :, idx].flatten()
    # if i == 1:
    #    true_full = np.log10(true_full)
    #    yy = np.log10(yy)
    #    yy = np.clip(yy, -6, None)
    #    true_full = np.clip(true_full, -6, None)
    # if i == 2:
    #    yy = 10**yy
    #    true_full = 10**true_full
    pparams = {}
    if param == "beta":
        pparams = dict(extent=(-2.8, 2, -2.8, 2))
        ax.set_xlim(-2.8, 0)
        ax.set_ylim(-2.8, 0)
    if param == "log10_floor_sfr_10":
        # Remove data below -3
        mask = (true_full > -3) & (yy > -3)
        true_full = true_full[mask]
        yy = yy[mask]
        mask = (ltrue_full > -3) & (lyy > -3)
        ltrue_full = ltrue_full[mask]
        lyy = lyy[mask]
    if param == "log10_Av":
        mask = (true_full > -2) & (yy > -2)
        true_full = true_full[mask]
        yy = yy[mask]
        mask = (ltrue_full > -2) & (lyy > -2)
        ltrue_full = ltrue_full[mask]
        lyy = lyy[mask]

    ax.hexbin(
        true_full,
        yy,
        gridsize=50,
        cmap="cmr.sunburst_r",
        mincnt=1,
        **pparams,
    )  # bins='log')
    ax.set(adjustable="box", aspect="equal")

    """ax.hexbin(
        ltrue_full,
        lyy,
        gridsize=50,
        cmap="Blues",
        mincnt=1,
        alpha=0.5,
        **pparams,
    )  # bins='log')
"""
    if i > 2:
        ax.set_xlabel("True Value")
    if i == 0 or i == 3:
        ax.set_ylabel("Inferred Value")
    ax.plot(
        [true_full.min(), true_full.max()], [true_full.min(), true_full.max()], "k--", alpha=0.5
    )
    ax.text(0.05, 0.90, f"${param_latex[i]}$", transform=ax.transAxes, fontsize=11)
    ax.text(0.05, 0.80, "RMSE = {:.2f}".format(rmse), transform=ax.transAxes, fontsize=10)
    ax.text(0.05, 0.70, f"$R^2={r2:.2f}$", transform=ax.transAxes, fontsize=10)
plt.tight_layout()
plt.subplots_adjust(hspace=0.2, wspace=-0.25)
fig.savefig(
    f"/home/tharvey/work/synference/models/{model_name}/plots/{model_name}_{extra}_validation.png",
    dpi=300,
    bbox_inches="tight",
)
fig.savefig(f"plots/{model_name}_{extra}_validation.png", dpi=300, bbox_inches="tight")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Make text bigger

plt.rcParams.update({"font.size": 12, "axes.labelsize": 14, "axes.titlesize": 14})


# CORRECTED FUNCTION
def calculate_quantile_error_per_param(
    true_params: np.ndarray, posteriors: np.ndarray
) -> np.ndarray:
    """Calculates a quantile-based error metric for each parameter of each observation.

    This version is corrected to handle posteriors with the shape
    (n_samples, n_points, n_dim), as indicated by the traceback.
    """
    # Your shapes:
    # true_params: (n_points, n_dim) -> (25000, 11)
    # posteriors:  (n_samples, n_points, n_dim) -> (1000, 25000, 11)

    # 1. Reshape true_params to (1, n_points, n_dim) to enable broadcasting.
    # This aligns the `n_points` and `n_dim` axes correctly.
    true_params_reshaped = true_params[np.newaxis, :, :]

    # 2. Perform the comparison. Broadcasting now works:
    # (1000, 25000, 11) < (1, 25000, 11) -> a boolean array of (1000, 25000, 11)
    comparison = posteriors < true_params_reshaped

    # 3. Take the mean over the `n_samples` axis (which is now axis=0).
    # This collapses the samples dimension, leaving a result of shape (n_points, n_dim).
    quantiles = np.mean(comparison, axis=0)

    errors = np.abs(quantiles - 0.5)

    return errors


def plot_colored_corner(
    data: np.ndarray,
    errors: np.ndarray,
    param_names: list[str] | None = None,
    elabel: str = "Prediction Error (|quantile - 0.5|)",
    params_to_remove: list[str] = [],
) -> None:
    """Creates a corner plot where scatter points are colored by the error metric."""
    n_dim = data.shape[1]
    # Remove specified parameters
    if params_to_remove:
        indices_to_keep = [i for i, name in enumerate(param_names) if name not in params_to_remove]
        data = data[:, indices_to_keep]
        errors = errors
        if param_names is not None:
            param_names = [param_names[i] for i in indices_to_keep]
        n_dim = data.shape[1]
    if param_names is None:
        param_names = [f"Param {i}" for i in range(n_dim)]

    fig, axes = plt.subplots(n_dim, n_dim, figsize=(12, 12))

    # Adjust subplots to be tight
    fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, hspace=0.1, wspace=0.1)

    for i in range(n_dim):
        for j in range(n_dim):
            ax = axes[i, j]

            # Diagonal: Plot 1D histograms of the true parameter distribution
            if i == j:
                sns.histplot(
                    data[:, i],
                    ax=ax,
                    bins=20,
                    kde=True,
                    element="step",
                    color="sandybrown",
                    alpha=0.7,
                )
                ax.set_ylabel("")
            # Hide upper triangle
            elif i < j:
                ax.axis("off")
            # Lower triangle: Plot 2D scatter plots
            else:
                # Color the points by the error metric
                """sc = ax.scatter(
                    data[:, j],
                    data[:, i],
                    cmap="cmr.ember",
                    vmin=np.percentile(errors, 5),
                    vmax=np.percentile(errors, 95),
                    s=5,
                    alpha=0.7,
                    c=errors,
                )"""
                # do a hexbin - not density just average error in each bin
                sc = ax.hexbin(
                    data[:, j],
                    data[:, i],
                    C=errors,
                    reduce_C_function=np.mean,
                    gridsize=25,
                    cmap="cmr.ember",
                    vmin=np.percentile(errors, 5),
                    vmax=np.percentile(errors, 95),
                    mincnt=1,
                )
            # Labels for the outer plots only
            if j == 0 and i > 0:
                ax.set_ylabel(param_names[i])
            if i == n_dim - 1:
                ax.set_xlabel(param_names[j])

            # Remove ticks for inner plots to clean up the view
            if i < n_dim - 1:
                ax.tick_params(axis="x", which="both", bottom=False, top=False, labelbottom=False)
            if j > 0:
                ax.tick_params(axis="y", which="both", left=False, right=False, labelleft=False)

    # Add a colorbar
    cbar_ax = fig.add_axes([0.35, 0.7, 0.55, 0.02])
    cbar = fig.colorbar(sc, cax=cbar_ax, orientation="horizontal")
    cbar.set_label(elabel)

    # plt.suptitle("Corner Plot of True Parameters Colored by Prediction Error", fontsize=16, y=0.95) # noqa E501
    plt.show()


sample_path = (
    f"/home/tharvey/work/synference/models/{model_name}/plots/{extra}/posterior_samples.npy"
)
true = fitter._y_test

samples = np.load(sample_path)
log_prob_path = f"/home/tharvey/work/synference/models/{model_name}/plots/{extra}/true_logprobs.npy"

log_probs = np.load(log_prob_path)

errors = calculate_quantile_error_per_param(true, samples)
errors = np.mean(errors, axis=1)

params_to_remove = ["$\\beta$"]
plot_colored_corner(
    true,
    log_probs,
    param_names=param_labels,
    elabel="$\\log P(\\theta | x)$",
    params_to_remove=params_to_remove,
)

## Pick a few examples and compare posteriors with nested sampling and the truth

In [ ]:
fitter.recreate_simulator_from_grid();

In [ ]:
import operator as op
from typing import List, Tuple, Union

import numpy as np


def search_parameter_array(
    array: np.ndarray,
    parameter_names: List[str],
    constraints: List[Tuple[str, str, Union[int, float]]],
) -> np.ndarray:
    """Return indexes in array with columns which meet constraints.

    Args:
        array (np.ndarray): The data array where rows are entries and columns
                            are parameters.
        parameter_names (List[str]): A list of string names for each column
                                     in the array.
        constraints (List[Tuple[str, str, Union[int, float]]]):
            A list of tuples, where each tuple defines a constraint in the
            format: (parameter_name, operator_string, value).
            e.g., ('mass', '>', 100)

    Returns:
        np.ndarray: A NumPy array of integer indices for the rows in the input
                    array that satisfy all of the given constraints.
    """
    # A mapping from string representations of operators to the actual
    # functions from the `operator` module.
    operator_map = {">": op.gt, "<": op.lt, ">=": op.ge, "<=": op.le, "==": op.eq, "!=": op.ne}

    # Start with a boolean mask of all True values, the same length as the array.
    # We will combine this with the mask from each constraint.
    combined_mask = np.ones(array.shape[0], dtype=bool)

    # Iterate over each constraint and apply it to the array
    for param_name, operator_str, value in constraints:
        try:
            # Find the column index for the given parameter name
            col_idx = parameter_names.index(param_name)
        except ValueError:
            print(f"Warning: Parameter '{param_name}' not found. Skipping constraint.")
            continue

        # Get the corresponding operator function from our map
        op_func = operator_map.get(operator_str)
        if not op_func:
            raise ValueError(f"Unsupported operator: {operator_str}")

        # Create a boolean mask for the current constraint
        current_mask = op_func(array[:, col_idx], value)

        # Update the combined mask using a logical AND.
        # This ensures that only rows satisfying *all* constraints are kept.
        combined_mask &= current_mask

    # np.where returns a tuple of arrays (one for each dimension).
    # Since our mask is 1D, we just need the first element.
    return np.where(combined_mask)[0]

In [ ]:
fitter.fitted_parameter_names

In [ ]:
indices

In [ ]:
indices = [38304, 14238, 3448, 87566, 92980, 61720, 98859, 56103]

# 86137 - for last

In [ ]:
# Make a gridspec with 2x4 plots

arr = copy.deepcopy(fitter._y_test)
constraint_names = copy.deepcopy(list(fitter.fitted_parameter_names))

redshift = copy.deepcopy(fitter._X_test[:, fitter.feature_names.index("redshift")])
arr = np.column_stack((arr, redshift))
constraint_names.append("redshift")

flexoki_colors = [
    "#D14D41",
    "#DA702C",
    "#D0A215",
    "#879A39",
    "#3AA99F",
    "#4385BE",
    "#8B7EC8",
    "#CE5D97",
]

fig = plt.figure(figsize=(7, 10), dpi=150)
ax = fig.subplots(nrows=4, ncols=2, sharex="col")
fig.subplots_adjust(wspace=0.01, hspace=0.01)
ax = ax.flatten()
names = [
    "Dusty $z=2$ starburst",
    "Massive $z=0.1$ quiescent",
    "Post-Starburst at $z=5$",
    "Highly Star-forming at $z=7$",
    "Starforming at $z=13$",
    "Dusty $z=7$ starburst",
    "Low-mass $z=2$ starforming",
    "Low-mass $z=9$ starforming",
]


generate_new = [False, False, False, False, False, False, False, True]

constraints = [
    [
        ("log_mass", ">", 9.5),
        ("log10_Av", ">", 0.5),
        ("log10_floor_sfr_10", ">", 1),
        ("redshift", "<", 2.5),
        ("redshift", ">", 1.5),
    ],
    [
        ("log_mass", ">", 10.5),
        ("log10_Av", "<", 0.5),
        ("log10_floor_sfr_10", "<", -1),
        ("redshift", "<", 0.5),
        ("redshift", ">", 0.0),
    ],
    [
        ("log_mass", ">", 9.5),
        ("log10_Av", "<", 0.5),
        ("log10_floor_sfr_10", "<", -1),
        ("log10_mass_weighted_age", "<", 2.5),
        ("redshift", "<", 6),
        ("redshift", ">", 4),
    ],
    [
        ("log_mass", ">", 8.5),
        ("log10_Av", "<", 0.5),
        ("log10_floor_sfr_10", ">", 1),
        ("redshift", "<", 8),
        ("redshift", ">", 6),
    ],
    [
        ("log_mass", ">", 8.5),
        ("log10_Av", "<", 0.5),
        ("log10_floor_sfr_10", ">", -1),
        ("redshift", "<", 14),
        ("redshift", ">", 12),
    ],
    [
        ("log_mass", ">", 9.5),
        ("log10_Av", ">", 0.5),
        ("log10_floor_sfr_10", ">", 1),
        ("redshift", "<", 8),
        ("redshift", ">", 6),
    ],
    [
        ("log_mass", "<", 8.5),
        ("log10_Av", "<", 0.5),
        ("log10_floor_sfr_10", ">", -1),
        ("redshift", "<", 3),
        ("redshift", ">", 1),
    ],
    [
        ("log_mass", "<", 8.5),
        ("log10_Av", "<", 0),
        ("log10_floor_sfr_10", ">", -1),
        ("redshift", "<", 9.5),
        ("redshift", ">", 8.5),
    ],
]

for i in range(len(names)):
    if i == 0:
        ax[i].text(
            0.98, 0.95, names[i], ha="right", va="top", transform=ax[i].transAxes, fontsize=10
        )
    else:
        ax[i].text(
            0.98, 0.02, names[i], ha="right", va="bottom", transform=ax[i].transAxes, fontsize=10
        )
    if i % 2 == 0:
        ax[i].set_ylabel(r"$\rm F _\lambda ~ [arb. units]$", fontsize=10)

    try:
        constraint = constraints[i]
        indexes = search_parameter_array(arr, constraint_names, constraint)
        print(f"Found {len(indexes)} matches for {names[i]}")

        if len(indexes) == 0:
            continue

        if generate_new[i]:
            rand_index = np.random.choice(indexes, size=1)[0]
        else:
            rand_index = indices[i]

        indices[i] = rand_index
        fitter.recover_SED(
            plot=True,
            ax=ax[i],
            X_test=fitter._X_test[rand_index, :],
            true_parameters=fitter._y_test[rand_index, :],
            num_samples=250,
            phot_unit="erg/s/cm^2/Angstrom",
            sample_color=flexoki_colors[i],
        )
    except IndexError:
        continue
    # remove legend
    ax[i].legend_.remove()
    ax[i].set_yscale("log")
    ax[i].set_ylim(ax[i].get_ylim()[1] * 1e-3, 1.5 * ax[i].get_ylim()[1])
    # Deal with limits being linked between rows - only adjust if needed
    # if i >= 2:
    #    actual_max = max(ax[i].get_ylim()[1], ax[i-2].get_ylim()[1])
    #    ax[i].set_ylim(actual_max*1e-4, actual_max)
    # Disable ytick labels for all
    ax[i].set_yticklabels([])

ax[6].set_xlabel(r"Obs. Wavelength [$\mu m$]", fontsize=10)
ax[7].set_xlabel(r"Obs. Wavelength [$\mu m$]", fontsize=10)

# Change xticklabels on bottom to be in microns
xticks = ax[6].get_xticks()
ax[6].set_xticklabels([f"{x / 1e4:.1f}" for x in xticks])
ax[7].set_xticklabels([f"{x / 1e4:.1f}" for x in xticks])

ax[0].legend(loc="lower right", fontsize=8)

fig.savefig(f"plots/{model_name}_{extra}_example_seds.png", dpi=300, bbox_inches="tight")

print(indices)

In [ ]:
ax[2].get_ylim()

In [ ]:
ax[-1].set_ylim(3e-23, ax[-1].get_ylim()[1])
ax[1].set_ylim(7e-20, ax[1].get_ylim()[1])
ax[4].set_ylim(1e-21, ax[4].get_ylim()[1])
ax[2].set_ylim(3e-21, ax[2].get_ylim()[1])
ax[3].set_ylim(3e-21, ax[3].get_ylim()[1])
fig.savefig(f"plots/{model_name}_{extra}_example_seds.png", dpi=300, bbox_inches="tight")

fig

In [ ]:
import astropy.units as u

wav = 0.9 * u.micron
flux = 28.542376 * u.ABmag

flux.to(u.erg / u.s / u.cm**2 / u.AA, equivalencies=u.spectral_density(wav))
flux.to(u.ABmag)

In [ ]:
from unyt import unyt_quantity

a = unyt_quantity(1.0, "nJy").to_astropy()

In [ ]:
constraints = [("log_mass", ">", 10), ("log10_mass_weighted_age", ">", 3), ("log10_Av", ">", 0.5)]
indexes = search_parameter_array(fitter._y_test, list(fitter.fitted_parameter_names), constraints)

indexes

In [ ]:
fitter._X_test[3435]

indexes = 99, 3435, 16, 9985, 7372

In [ ]:
fitter._X_test[99]

In [ ]:
for index in indices:
    sample = fitter._X_test[index]
    true_params = fitter._y_test[index]
    predicted_params = fitter.sample_posterior(sample, num_samples=1000)

    fitter._prior = fitter.create_priors()

    result = fitter.fit_observation_using_sampler(
        observation=sample,
        sampler="dynesty",
        sampler_kwargs={"bound": "multi", "sample": "rwalk", "run_kwargs": {"n_effective": 2000}},
        remove_params=["log10_mass_weighted_age", "log10_floor_sfr_10"],
        plot_name=f"{model_name}_{extra}_example_{index}",
        min_flux_pc_error=0.05,
    )
    samples = result["samples"]
    log_l = result["logl"]
    log_w = result["logwt"]

    data = {
        "samples": samples,
        "log_weights": log_w,
        "log_likelihood": log_l,
    }

    np.savez(f"plots/{model_name}_{extra}_example_{index}_dynesty.npz", **data)

# Plot corner of both on same (see through, only contours) and true value

In [ ]:
?DynamicNestedSampler

In [ ]:
import corner

comparison_params = [
    "log_mass",
    "log10metallicity",
    "log10_Av",
    "log_sfr",
    "sfh_quantile_25",
    "sfh_quantile_50",
    "sfh_quantile_75",
]

display_names = [
    r"$\log M_*$",
    r"$\log Z/Z_\odot$",
    "$A_V$",
    r"$\log$ SFR",
    "$t_{25}$",
    "$t_{50}$",
    "$t_{75}$",
]

mask_idx = [True if p in comparison_params else False for p in fitter.fitted_parameter_names]
mask_idx = np.array(mask_idx, dtype=bool)

figure = corner.corner(
    points[:, :-2],
    weights=np.exp(log_w - log_w.max()),
    labels=display_names,
    color="#03A89E",
    plot_datapoints=False,
    fill_contours=False,
    contour_kwargs={"linewidths": 1.5, "linestyles": "solid"},
    contourf_kwargs={
        "alpha": 0.3  # Very light fill for contours
    },
    hist_kwargs={"alpha": 0.7, "histtype": "stepfilled", "edgecolor": "black", "linewidth": 1},
    levels=(0.68, 0.95),
    plot_density=False,
    smooth=1.0,
    label_kwargs={"fontsize": 22},
    title_kwargs={"fontsize": 22},
    quantiles=[0.16, 0.5, 0.84],
    show_titles=False,
    # Increase tick label size
    tick_label_kwargs={"fontsize": 16},
)

corner.corner(
    predicted_params[:, mask_idx],
    labels=display_names,
    color="#CD3700",
    plot_datapoints=False,
    fill_contours=False,
    contour_kwargs={"linewidths": 1.5, "linestyles": "solid"},
    contourf_kwargs={
        "alpha": 0.3  # Very light fill for contours
    },
    hist_kwargs={"alpha": 0.7, "histtype": "stepfilled", "edgecolor": "black", "linewidth": 1},
    plot_density=False,
    smooth=1.0,
    fig=figure,
    quantiles=[0.16, 0.5, 0.84],
    show_titles=False,
    tick_label_kwargs={"fontsize": 16},
)

# Add true value lines
ndim = len(comparison_params)
axes = np.array(figure.axes).reshape((ndim, ndim))
for i in range(ndim):
    for j in range(ndim):
        ax = axes[i, j]
        # Increase axis line weight
        for axis in ["top", "bottom", "left", "right"]:
            ax.spines[axis].set_linewidth(1.5)
        if i == j:
            ax.axvline(true_params[i], color="k", linestyle="dotted")
            # Add our own titles with quantiles for both
            samples_param = predicted_params[:, mask_idx][:, i]
            p16, p50, p84 = np.percentile(samples_param, [16, 50, 84])
            nested = points[:, :-2][:, i]
            # need to get weighted percentiles for nested sampling
            sorted_indices = np.argsort(nested)
            nested_sorted = nested[sorted_indices]
            weights_sorted = np.exp(log_w - log_w.max())[sorted_indices]
            cumulative_weights = np.cumsum(weights_sorted)
            cumulative_weights /= cumulative_weights[-1]  # Normalize to [0, 1]
            p16_ns = np.interp(0.16, cumulative_weights, nested_sorted)
            p50_ns = np.interp(0.50, cumulative_weights, nested_sorted)
            p84_ns = np.interp(0.84, cumulative_weights, nested_sorted)
            title = (
                rf"{comparison_params[i]}" + "\n"
                rf"SBI: ${p50:.2f}_{{-{p50 - p16:.2f}}}^{{+{p84 - p50:.2f}}}$" + "\n"
                rf"NS: ${p50_ns:.2f}_{{-{p50_ns - p16_ns:.2f}}}^{{+{p84_ns - p50_ns:.2f}}}$"
            )
            # ax.set_title(title, fontsize=14)
            ax.set_title("")
            # Approximate the title with textbox so we can set the color of SBI vs NS
            sbi = rf"SBI: ${p50:.2f}_{{-{p50 - p16:.2f}}}^{{+{p84 - p50:.2f}}}$"
            ns = rf"NS: ${p50_ns:.2f}_{{-{p50_ns - p16_ns:.2f}}}^{{+{p84_ns - p50_ns:.2f}}}$"

            ax.text(
                0.5,
                1.3,
                sbi,
                color="#CD3700",
                fontsize=22,
                ha="center",
                va="center",
                transform=ax.transAxes,
                fontweight="bold",
            )
            ax.text(
                0.5,
                1.1,
                ns,
                color="#03A89E",
                fontsize=22,
                ha="center",
                va="center",
                transform=ax.transAxes,
                fontweight="bold",
            )

        elif i > j:
            ax.axvline(true_params[j], color="k", linestyle="dotted")
            ax.axhline(true_params[i], color="k", linestyle="dotted")
            ax.plot(true_params[j], true_params[i], "ks")

        # Increase font size of tick labels if i or j is 0 or ndim-1
        if i == ndim - 1 or j == 0:
            ax.tick_params(axis="x", which="major", labelsize=16)
        if j == 0 or i == ndim - 1:
            ax.tick_params(axis="y", which="major", labelsize=16)

# figure.suptitle("Comparison of Posterior Samples: MDN vs Nested Sampling", fontsize=16)

# Add upper right floating axis for SED plot

new_ax = figure.add_axes([0.50, 0.65, 0.45, 0.35])

sfh_ax = figure.add_axes([0.75, 0.42, 0.22, 0.18])

# Get SEDs given these posterior samples.
results = fitter.recover_SED(
    sample,
    samples=predicted_params,
    plot=True,
    true_parameters=true_params,
    sample_color="#CD3700",
    fig=figure,
    ax=new_ax,
    plot_histograms=False,
    plot_sfh=True,
    ax_sfh=sfh_ax,
    num_samples=50,
)

# Do the same for nested sampling samples
# add two dimensions to fake points to match expected shape

points_dict = {name: points[:, i] for i, name in enumerate(comparison_params)}
# Not designed to deal with weights, so sample with replacement according to weights
weights = np.exp(log_w - log_w.max())
weights /= weights.sum()
indices = np.random.choice(len(weights), size=len(weights), replace=True, p=weights)
points_dict = {name: points_dict[name][indices] for name in points_dict}
results_ns = fitter.recover_SED(
    sample,
    samples=points_dict,
    plot=True,
    sample_color="#03A89E",
    fig=figure,
    ax=new_ax,
    plot_histograms=False,
    plot_sfh=True,
    ax_sfh=sfh_ax,
    num_samples=50,
)

new_ax.set_ylim(25, 22)
# move legend to lower right

# Increase font size of all text in new_ax and sfh_ax
for item in (
    new_ax.title,
    new_ax.xaxis.label,
    new_ax.yaxis.label,
    new_ax.get_xticklabels(),
    new_ax.get_yticklabels(),
    new_ax.get_legend().get_texts(),
    sfh_ax.title,
    sfh_ax.xaxis.label,
    sfh_ax.yaxis.label,
    sfh_ax.get_xticklabels(),
    sfh_ax.get_yticklabels(),
):
    if hasattr(item, "__iter__"):
        for subitem in item:
            subitem.set_fontsize(16)
    else:
        item.set_fontsize(16)

# Increase marker size for all markers in new_ax
for line in new_ax.get_lines():
    line.set_markersize(8)

# Change line labels in new_ax to "SBI Posterior" and "Nested Sampling"
lines = new_ax.get_lines()
for line in lines:
    if (
        line.get_color() == "#CD3700"
        and line.get_label() != "_nolegend_"
        and "_child" not in line.get_label()
    ):
        line.set_label("Predicted SBI SED")
    elif (
        line.get_color() == "#03A89E"
        and line.get_label() != "_nolegend_"
        and "_child" not in line.get_label()
    ):
        print(line.get_label())
        line.set_label("Predicted NS SED")


# Find labelled markers and rename them
for marker in new_ax.collections + new_ax.lines:
    if not hasattr(marker, "get_label"):
        continue
    if marker.get_label() == "Posterior Phot." and marker.get_color() == "#03A89E":
        marker.set_label("Predicted NS Phot.")
    elif marker.get_label() == "Posterior Phot." and marker.get_color() == "#CD3700":
        marker.set_label("Predicted SBI Phot.")

new_ax.legend(loc="lower right", fontsize=14)

figure.savefig(
    f"plots/posterior_comparison_{index}.png",
    dpi=300,
    bbox_inches="tight",
)

### 5 SED recoveries and true SEDs (different redshifts, masses, dust etc)

In [ ]:
test_samples = []
for test_sample in test_samples:
    fitter.recover_SED(X_test=test_sample)

### SFH Recovery Examples

In [ ]:
np.unique(fitter.simulator.unused_params)

In [ ]:
random_idx = np.random.randint(0, len(fitter._X_test), size=100)

for idx in random_idx:
    sample = fitter._X_test[idx]
    true_params = fitter._y_test[idx]
    predicted_params = fitter.recover_SED(
        sample, num_samples=50, plot=True, true_parameters=true_params
    )

In [ ]:
# Sort indexes for high log10_Av and low redshift
Av_idx = list(fitter.fitted_parameter_names).index("log10_Av")
z_idx = list(fitter.feature_names).index("redshift")
mask = (fitter._y_test[:, Av_idx] > 0.5) & (fitter._X_test[:, z_idx] < 0.5)

for i in range(len(fitter._X_test[mask])):
    s = fitter._X_test[mask][i]
    true_params = fitter._y_test[mask][i]
    fitter.recover_SED(s, num_samples=50, plot=True, true_parameters=true_params)

# Benchmark Inference Speed

In [ ]:
# Test speed of normal model.


fitter = SBI_Fitter.load_saved_model(
    f"/home/tharvey/work/synference/models/{model_name}/{model_name}_{extra}_posterior.pkl",
    grid_path="/home/tharvey/work/synference/grids/grid_BPASS_Chab_DenseBasis_SFH_0.01_z_14_logN_6.0_Calzetti_v4_multinode.hdf5",
    device="cuda",
)
X_test = fitter._X_test[:1000]

out = fitter.sample_posterior(X_test, num_samples=1000, log_times=True)